# Matching Engine for Agents, JPM paper (in progress...)
[Reinforcement Learning for Market Making in a Multi-agent Dealer Market](https://arxiv.org/abs/1911.05892)

### TODO:
- Pnl from inventory mtm not showing.
- Implement agent hedging.
- Use LOB data structure for blind auctions.
- Implement Reinforcement Learning agent.

In [3]:
import numpy as np
import scipy.stats as stats
import pandas as pd
import bisect

# =======================================
# ==========  exchange.py ===============
# =======================================
# Reference market, stream prices relative this. Also to be used for hedging
class AssetDynamics: # TODO: rename to e.g. MarketDynamics
    def __init__(self, s0, dt, mu, vol):
        self.s = s0
        self.dt = dt
        self.mu = mu
        self.vol = vol
        self.t = 0
        self.bid_spread = 2 / 10000
        self.ask_spread = 2 / 10000
    def evolve(self):
        self.s *= np.exp(self.mu * self.dt - 0.5*self.vol**2*self.dt + self.vol*self.dt**0.5*np.random.normal())
        self.t += self.dt
    def bid_price(self):
        return self.s *(1 + self.bid_spread)
        
    def ask_price(self):
        return self.s *(1 + self.ask_spread)
        
# =======================================
# ==========  accounting.py =============
# =======================================
class Pnl:
    def __init__(self):
        self.spread = 0
        self.inventory = 0
        self.hdg = 0
    def total_pnl(self):
        return self.spread + self.inventory + self.hdg
    def __str__(self):
        return 'spread: {}, inventory: {}, hdg: {}'.format(self.spread, self.inventory, self.hdg)
        
class Inventory:
    def __init__(self):
        self.units = 0
    def __str__(self):
        return 'inventory: {}'.format(self.units)
        
# =======================================
# ==========  lob.py =============
# =======================================
class RFQ:
    def __init__(self, sign, size, investor):
        self.sign = sign
        self.size = size
        self.investor = investor
        
class Transaction:
    def __init__(self, sn, time, agent, investor, sign, size, price, spread):
        self.sn = sn # serial number
        self.time = time
        self.agent = agent
        self.investor = investor
        self.sign = sign
        self.size = size
        self.price = price
        self.spread = spread

class OBEntry:
    def __init__(self, price, size, agent):
        self.price = price
        self.size = size
        self.agent = agent
if 0:
    class OrderBook:
        def __init__(self, security):
            self.security = security
            self.bids = []
            self.asks = []
        def stream_bid(self, price, size, agent):
            self.bids.append(OBEntry(price, size, agent))

        def stream_ask(self, price, size, agent):
            self.asks.append(OBEntry(price, size, agent))

        def stream_prices(self, obentries : list):
            b = filter(entries, lambda e :  e.size > 0)
            self.bids.extend(b)
            a = filter(entries, lambda e :  e.size < 0)
            self.asks.extend(a)

        def clear_streams():
            self.bids = []
            self.asks = []

        def RFQ_transact(rfq):
            #self.bids.sort(key=lambda entry : entry.price)
            #self.asks.sort(key=lambda entry : entry.price)
            matched_order = None
            if rfq.sign == 1:
                mathed_order = min(self.asks, key=lambda entry : entry.price)
            elif rfq.sign == -1:
                mathed_order = max(self.bids, key=lambda entry : entry.price)
            else:
                raise Exception()

            if matched_order:
                Transaction(sn=sn, time=self.t, agent=matched_order.agent, investor=rfq.investor, sign=rfq.sign, size=rfq.size, price=bo_price, spread=bo_spr)
    
# =======================================
# ==========  engine.py =================
# =======================================
class Engine:
    def __init__(self):
        self.exchange = None
        self.investors = []
        self.agents = []
        self.transactions = []
    def set_exchange(self, exchange):
        self.exchange = exchange
    def add_investor(self, investor):
        self.investors.append(investor)
    def add_agent(self, agent):
        self.agents.append(agent)
        
    def transactions_df(self):
        fields = ['sn', 'time', 'agent', 'investor', 'sign', 'size', 'price', 'spread']
        #tmp = [{'sn': getattr(f, 'sn')} for f in self.transactions]
        df = pd.DataFrame([{fn: getattr(f, fn) for fn in fields} for f in self.transactions])
        df.rename(inplace=True, columns={'spread':'spread_bp'}) 
        df['spread_bp'] = df['spread_bp']*10000
        return df
        
    def evolve(self):
        self.exchange.evolve()
    
    def execute_match(self):
        # Assume agents quote infinite size. TODO: implement diff sizes.
        bo_sprds = [a.quote_spreads(self.exchange) for a in self.agents]
        offer_sprds, bid_sprds = zip(*bo_sprds)
        bb_idx = np.argmin(bid_sprds)
        bo_idx = np.argmin(offer_sprds)
        bb_spr, bb_agent = bid_sprds[bb_idx], self.agents[bb_idx]
        bo_spr, bo_agent = bid_sprds[bo_idx], self.agents[bo_idx]
        bb_price = self.exchange.s - bb_spr
        bo_price = self.exchange.s + bo_spr
        
        # Investors
        trans = []
        sn = len(self.transactions)
        for i in self.investors:
            trd = i.gentrade()
            if trd:
                if trd.sign > 0: #Investor buys
                    tmp = Transaction(sn=sn, time=self.exchange.t, agent=bo_agent, investor=i, sign=-trd.sign, size=trd.size, price=bo_price, spread=bo_spr)
                elif trd.sign < 0: #Investor sells
                    tmp = Transaction(sn=sn, time=self.exchange.t, agent=bo_agent, investor=i, sign=-trd.sign, size=trd.size, price=bb_price, spread=bb_spr)
                else:
                    raise Exception()
                trans.append(tmp)
                sn += 1
                
        # execute (pnl)
        for t in trans:
            t.agent.inventory.units += -t.sign * t.size
            t.agent.pnl.spread += t.spread * t.size
            t.agent.transactions.append(t)
            
        self.transactions.extend(trans)
                
            
# =======================================
# ==========  investors.py ==============
# =======================================
# Liquidity takers of streamed prices.
class Investor:
    id_counter = 0
    def __init__(self):
        self.uid = Investor.id_counter
        Investor.id_counter += 1
        arrival_intensity = 1
        self.arrival_dist = stats.poisson(arrival_intensity)
        buy_prob = 0.5
        self.dir_dist = stats.bernoulli(buy_prob)
        size_rate = 0.5 # lambda
        self.size_dist = stats.expon(scale=1/size_rate)
    def gentrade(self):
        if self.arrival_dist.rvs():
            sign = 2*self.dir_dist.rvs() - 1
            size = np.ceil(self.size_dist.rvs())
            return RFQ(sign, size, self)
        else:
            return None
    def __str__(self):
        return '{} {}'.format(type(self).__name__, self.uid)
        
# =======================================
# ==========  agents.py =================
# =======================================
# Agents streaming prices
class Agent:
    def __init__(self):
        self.inventory = Inventory()
        self.pnl = Pnl()
        self.transactions = []
    def quote_spreads():
        raise NotImplementedError()
        
class RandomAgent(Agent):
    id_counter = 0
    def __init__(self, eps_min, eps_max):
        Agent.__init__(self)
        self.uid = RandomAgent.id_counter
        RandomAgent.id_counter += 1
        self.eps_buy_dist = stats.uniform(loc=eps_min, scale=eps_max - eps_min)
        self.eps_sell_dist = stats.uniform(loc=eps_min, scale=eps_max - eps_min)
        self.hdgfrac_dist = stats.uniform(loc=0, scale=1)
    def quote_spreads(self, exchange):
        # Agent's streamed bid
        si_bid = exchange.bid_spread * (1 + self.eps_buy_dist.rvs())
        
        # Agent's streamed bid
        si_ask = exchange.ask_spread * (1 + self.eps_sell_dist.rvs())
        return (si_bid, si_ask)
    
    def __str__(self):
        return type(self).__name__ + ' ' + str(self.uid)
        
class PersistentAgent(Agent):
    id_counter = 0
    def __init__(self, eps_buy, eps_sell, hdgfrac):
        Agent.__init__(self)
        self.uid = RandomAgent.id_counter
        RandomAgent.id_counter += 1
        self.eps_buy = eps_buy
        self.eps_sell = eps_sell
        self.hdgfrac = hdgfrac
    def quote_spreads(self, exchange):
        si_bid = exchange.bid_spread * (1 + self.eps_buy)
        si_ask = exchange.ask_spread * (1 + self.eps_sell)
        return (si_bid, si_ask)
    
    def __str__(self):
        return type(self).__name__ + ' ' + str(self.uid)
        
        
# =======================================
# ============  main.py =================
# =======================================
invtrs = [Investor()]
ag1 = PersistentAgent(eps_buy=0.0, eps_sell=0.0, hdgfrac=0)
ag2 = RandomAgent(eps_min=-0.5, eps_max=0.5)
ag3 = RandomAgent(eps_min=-1.0, eps_max=1.0)
agnts = [ag1, ag2, ag3]

engine = Engine()
exchange = AssetDynamics(s0=100, dt=1, mu=0, vol=0.000000001)
engine.set_exchange(exchange)
for i in invtrs:
    engine.add_investor(i)
for a in agnts:
    engine.add_agent(a)

#matchingEngine = MatchingEngine()
# =======================================
# =============  RUN ====================
# =======================================
for i in range(1, 20):
    # 1. Each Agent Publishes prices
    engine.execute_match()
    engine.evolve()

    
# =======================================
# ============  RESULTS =================
# =======================================
for a in agnts:
    print('==========')
    print(str(a))
    print(a.inventory)
    print(a.pnl)

print(' ========================')
print(' ===== TRANSACTIONS =====')
print(engine.transactions_df().round({'price': 4, 'size':2, 'spread_bp':2}))


PersistentAgent 0
inventory: -1.0
spread: 0.0015992007411609263, inventory: 0, hdg: 0
RandomAgent 1
inventory: 1.0
spread: 0.0027255876835155933, inventory: 0, hdg: 0
RandomAgent 2
inventory: -3.0
spread: 0.0012179987921389073, inventory: 0, hdg: 0
 ===== TRANSACTIONS =====
    sn  time              agent    investor  sign  size     price  spread_bp
0    0     0      RandomAgent 1  Investor 0     1   5.0   99.9999       1.15
1    1     1      RandomAgent 2  Investor 0     1   1.0  100.0000       0.25
2    2     4      RandomAgent 1  Investor 0    -1   1.0  100.0003       2.78
3    3     5      RandomAgent 1  Investor 0     1   1.0   99.9998       2.00
4    4     6  PersistentAgent 0  Investor 0     1   2.0   99.9998       1.82
5    5     8      RandomAgent 1  Investor 0    -1   2.0  100.0002       2.04
6    6     9      RandomAgent 1  Investor 0    -1   3.0  100.0003       2.61
7    7    10      RandomAgent 1  Investor 0    -1   3.0  100.0001       1.09
8    8    11  PersistentAgent 0 